# LangChain 调用 MCP 详细使用教程

## 什么是 MCP (Model Context Protocol)？

MCP（Model Context Protocol，模型上下文协议）是一个开放标准协议，旨在规范大型语言模型 (LLM) 与外部工具、数据源和服务交互的方式。它由 Anthropic 提出，并被 LangChain 等框架采纳，以增强 AI 应用的能力。

MCP 的核心思想是创建一个客户端-服务器架构：

* **MCP 服务器 (MCP Server)**：这些服务器暴露一组可供 LLM 使用的工具或数据接口。例如，一个 MCP 服务器可以提供数学计算工具、文件系统操作工具、天气查询API接口等。
* **MCP 客户端 (MCP Client)**：通常是 LLM 应用或代理 (Agent)，它们连接到 MCP 服务器以使用其提供的工具来完成特定任务。

通过 MCP，开发者可以构建更强大、更具互操作性的 AI 系统，使得 LLM 能够安全、可控地访问和利用外部世界的信息与能力。

## LangChain 与 MCP 的集成

LangChain 提供了与 MCP 生态系统集成的能力，主要通过以下库：

* `mcp`: 用于创建 MCP 服务器的 Python 库（例如，使用 `FastMCP` 可以快速定义和运行 MCP 服务器）。
* `langchain-mcp-adapters`: 这个库使得 LangChain (特别是 LangGraph Agents) 能够作为 MCP 客户端，发现并使用 MCP 服务器上定义的工具。

本教程将引导你完成以下步骤：

1.  安装必要的库。
2.  创建一个简单的 MCP 服务器，该服务器提供一些基本工具。
3.  创建一个 LangChain 代理 (Agent)，该代理能够调用 MCP 服务器上的工具。
4.  运行并测试整个流程。


## 步骤 1: 安装必要的库

首先，你需要安装 LangChain 核心库、用于创建 MCP 服务器的库、MCP 适配器库以及一个 LLM 提供商的库（本例中我们将使用 `langchain-openai`，你可以根据需要替换）。

```bash
pip install langchain langchain-openai langchain-mcp-adapters mcp
```

In [1]:
!pip install langchain langchain-openai langchain-mcp-adapters mcp

  Using cached mcp-1.8.0-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB

## 步骤 2: 创建一个简单的 MCP 服务器
我们将创建一个 MCP 服务器，它提供两个简单的数学工具：add (加法) 和 multiply (乘法)。

创建一个名为 mcp_server.py 的 Python 文件，并添加以下内容：

In [7]:
!pip install nest_asyncio

In [2]:
# mcp_server.py
from mcp.server.fastmcp import FastMCP
import asyncio

# 创建一个 FastMCP 实例，并命名为 "MathServer"
mcp_server = FastMCP(name="MathServer", description="A simple server with math tools.")

# 定义一个工具：加法
@mcp_server.tool()
async def add(a: int, b: int) -> int:
    """
    Adds two integers.
    For example: add(a=5, b=3) would return 8.
    """
    print(f"[MCP Server] Received add request: a={a}, b={b}")
    result = a + b
    print(f"[MCP Server] Sending add response: {result}")
    return result

# 定义另一个工具：乘法
@mcp_server.tool()
async def multiply(a: int, b: int) -> int:
    """
    Multiplies two integers.
    For example: multiply(a=5, b=3) would return 15.
    """
    print(f"[MCP Server] Received multiply request: a={a}, b={b}")
    result = a * b
    print(f"[MCP Server] Sending multiply response: {result}")
    return result

async def main():
    # 运行 MCP 服务器，使用 stdio 作为传输方式
    # 这意味着服务器将通过标准输入/输出与客户端通信
    # 对于生产环境，你可能会使用 HTTP 或其他网络传输方式
    print("[MCP Server] Starting MathServer on stdio...")
    await mcp_server.run(transport="stdio")

# if __name__ == "__main__":
#     asyncio.run(main())

main()



<coroutine object main at 0x000001FBAE7C0040>

说明:

FastMCP: 提供了一种快速定义工具和运行服务器的方式。
@mcp_server.tool(): 装饰器将一个 Python 函数注册为 MCP 服务器上的一个可用工具。函数的类型注解和文档字符串 (docstring) 非常重要，因为它们会被 MCP 客户端用来理解如何调用该工具以及工具的功能。
transport="stdio": 这里我们使用 stdio (标准输入/输出) 作为服务器和客户端之间的通信方式。这意味着服务器进程将通过其标准输入接收请求，并通过标准输出发送响应。这对于本地测试很方便。在实际部署中，你可能会使用 HTTP 等网络协议。

## 步骤 3: 创建 LangChain 代理以调用 MCP 工具
现在，我们将创建一个 LangChain 代理，它能够连接到我们刚刚创建的 MathServer 并使用其提供的 add 和 multiply 工具。

创建一个名为 langchain_mcp_client.py 的 Python 文件，并添加以下内容：

In [4]:
!pip install langchain-mcp-adapters langgraph


   ------------- -------------------------- 2/6 [langgraph-sdk]
   ------------- -------------------------- 2/6 [langgraph-sdk]
   -------------------- ------------------- 3/6 [langgraph-checkpoint]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   ---------------------------------------- 6/6 [langgraph]



In [6]:
# langchain_mcp_client.py
import asyncio
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_mcp_adapters.langgraph import MCPClient

# 替换为你 MCP 服务器的启动命令和工作目录
# 由于我们使用 stdio，客户端需要启动服务器进程
MCP_SERVER_COMMAND = ["python", "mcp_server.py"] # 确保 mcp_server.py 在你的PYTHONPATH中或指定完整路径
MCP_SERVER_CWD = "." # 服务器的工作目录

async def main():
    print("[LangChain Client] Initializing...")

    # 1. 初始化 LLM
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    # 2. 初始化 MCPClient
    # MCPClient 会启动并管理 MCP 服务器进程 (如果使用 stdio)
    # 它会发现服务器提供的工具
    mcp_client = MCPClient(
        name="math_solver", # 给这个客户端实例一个名字
        cmd=MCP_SERVER_COMMAND,
        cwd=MCP_SERVER_CWD,
        # verbose=True # 取消注释以获取更多调试信息
    )

    print("[LangChain Client] Connecting to MCP server and fetching tools...")
    try:
        # 连接到服务器并获取工具
        # 对于 stdio 传输，这会启动服务器子进程
        await mcp_client.connect()
        mcp_tools = await mcp_client.get_tools()
    except Exception as e:
        print(f"[LangChain Client] Error connecting to MCP server or getting tools: {e}")
        await mcp_client.disconnect() # 清理
        return

    if not mcp_tools:
        print("[LangChain Client] No tools found from MCP server.")
        await mcp_client.disconnect()
        return

    print(f"[LangChain Client] Successfully fetched {len(mcp_tools)} tools from MCP server:")
    for tool in mcp_tools:
        print(f"  - Tool Name: {tool.name}, Description: {tool.description}")

    # 3. 创建 LangChain Agent
    # 定义提示 (Prompt)
    # 注意: 我们需要确保提示能够引导 LLM 使用提供的工具。
    # "agent_scratchpad" 用于存储代理的中间思考和行动。
    # "input" 是用户的原始输入。
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant that can use tools to perform calculations. Use the available tools to answer the user's questions."),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )

    # 创建 Tool Calling Agent
    # 这个代理类型擅长决定何时以及如何调用工具
    agent = create_tool_calling_agent(llm, mcp_tools, prompt)

    # 创建 Agent Executor
    # AgentExecutor 负责实际运行代理、调用工具并获取最终响应
    agent_executor = AgentExecutor(agent=agent, tools=mcp_tools, verbose=True)

    # 4. 调用代理
    print("\n[LangChain Client] Asking agent to solve 'What is 5 + 3?'")
    response1 = await agent_executor.ainvoke({"input": "What is 5 + 3?"})
    print(f"\n[LangChain Client] Agent Response 1: {response1['output']}")

    print("\n[LangChain Client] Asking agent to solve 'What is 7 multiplied by 4?'")
    response2 = await agent_executor.ainvoke({"input": "What is 7 multiplied by 4?"})
    print(f"\n[LangChain Client] Agent Response 2: {response2['output']}")

    print("\n[LangChain Client] Asking agent to solve 'What is (10 + 5) * 2?'")
    response3 = await agent_executor.ainvoke({"input": "What is (10 + 5) * 2? First add 10 and 5, then multiply the result by 2."})
    print(f"\n[LangChain Client] Agent Response 3: {response3['output']}")

    # 5. 清理
    # 断开与 MCP 服务器的连接 (对于 stdio，这会终止服务器子进程)
    print("[LangChain Client] Disconnecting from MCP server...")
    await mcp_client.disconnect()
    print("[LangChain Client] Done.")

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'langchain_mcp_adapters.langgraph'

说明:

MCP_SERVER_COMMAND: 定义了如何启动你的 MCP 服务器。由于 mcp_server.py 使用 stdio，LangChain 客户端 (MCPClient) 需要知道如何启动这个服务器脚本。
MCPClient: 这是 langchain-mcp-adapters 提供的类，用于与 MCP 服务器交互。
cmd 和 cwd: 指定了启动服务器的命令和工作目录。
mcp_client.connect(): 建立与服务器的连接。对于 stdio，它会启动 mcp_server.py 作为一个子进程。
mcp_client.get_tools(): 从连接的 MCP 服务器检索可用工具列表。这些工具会自动转换为 LangChain Tool 对象。
ChatOpenAI: 我们使用的 LLM 实例。
create_tool_calling_agent 和 AgentExecutor: 这些是 LangChain 中用于创建和运行能够使用工具的代理的标准组件。代理会接收用户输入，通过 LLM 决定是否以及如何使用 mcp_tools 中的工具，然后执行这些工具并返回最终答案。
agent_executor.ainvoke(): 异步调用代理来处理输入。
mcp_client.disconnect(): 关闭与 MCP 服务器的连接，并（在 stdio 模式下）终止服务器子进程。